In [1]:
import pandas as pd
import joblib
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print("Libraries Loaded Successfully! 🚀")

Libraries Loaded Successfully! 🚀


In [2]:
# --- CONFIGURATION ---
models_config = {
    "sms": {
        "path": "dataset/spam.csv",       
        "encoding": "latin-1",
        "sep": ",",
        "col_text": "v2",
        "col_label": "v1",
        "filename": "models/sms_spam_model.pkl"
    },
    "email": {
        "path": "dataset/spam.csv",       # Using SMS data for Email (Safe fallback)
        "encoding": "latin-1",
        "sep": ",",
        "col_text": "v2",
        "col_label": "v1",
        "filename": "models/email_spam_model.pkl"
    },
    "news": {
        "path": "dataset/WELFake_Dataset.csv", 
        "sep": ",",
        "col_text": "title",
        "col_label": "label",
        "filename": "models/fake_news_model.pkl"
    }
}
print("Configuration Set! ✅")

Configuration Set! ✅


In [3]:
print("🚀 Starting Fast Training...")

for key, config in models_config.items():
    print(f"\n------------------------------------------------")
    print(f"📡 Processing Model: {key.upper()}")
    
    if not os.path.exists(config['path']):
        print(f"❌ Error: File not found at {config['path']}")
        continue

    try:
        # 1. Load Data
        if "encoding" in config:
            df = pd.read_csv(config['path'], encoding=config['encoding'], sep=config['sep'])
        else:
            df = pd.read_csv(config['path'], sep=config['sep'])

        # --- THE SPEED FIX ---
        # If it is the huge NEWS dataset, only take the first 3000 rows.
        if key == 'news':
            df = df.iloc[:3000]
            print("   ⚠️ TRUNCATED data to 3,000 rows for speed!")
        # ---------------------

        # 2. Prepare Columns
        text_col = config['col_text']
        label_col = config['col_label']
        
        # SMS/Email Standardization
        if key in ['sms', 'email']:
            if 'v1' not in df.columns:
                df.columns = ['v1', 'v2', 'u1', 'u2', 'u3']
            df['label_num'] = df['v1'].map({'ham': 0, 'spam': 1})
            text_col = 'v2'

        # News Standardization
        if key == 'news':
            # WELFake: 1=Real, 0=Fake. We want 0=Safe, 1=Danger.
            df['label_num'] = df[label_col].map({1: 0, 0: 1})

        # 3. Clean
        df = df.dropna(subset=[text_col, 'label_num'])
        X = df[text_col].astype(str)
        y = df['label_num'].astype(int)
        
        # 4. Train (No Split needed for final model, helps speed)
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),
            ('svm', SVC(kernel='linear', probability=True))
        ])
        
        print(f"   ⏳ Training on {len(df)} rows...")
        pipeline.fit(X, y)
        
        # 5. Save
        joblib.dump(pipeline, config['filename'])
        print(f"   💾 Saved to {config['filename']}")

    except Exception as e:
        print(f"❌ Error: {e}")

print("\n✨ All Models Trained Successfully!")

🚀 Starting Fast Training...

------------------------------------------------
📡 Processing Model: SMS
   ⏳ Training on 5572 rows...
   💾 Saved to models/sms_spam_model.pkl

------------------------------------------------
📡 Processing Model: EMAIL
   ⏳ Training on 5572 rows...
   💾 Saved to models/email_spam_model.pkl

------------------------------------------------
📡 Processing Model: NEWS
   ⚠️ TRUNCATED data to 3,000 rows for speed!
   ⏳ Training on 2977 rows...
   💾 Saved to models/fake_news_model.pkl

✨ All Models Trained Successfully!
